<a href="https://colab.research.google.com/github/smartinternz02/SI-GuidedProject-515069-1688188147/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Set the path to the PlantDoc dataset
dataset_path = '/content/drive/MyDrive/Colab Notebooks/PlantDoc-Dataset'

# Set the image size for resizing
image_size = (224, 224)

# Set the batch size for training
batch_size = 32

# Set the number of training samples
train_samples = 8000

# Set the number of epochs for training
epochs = 10

# Update the data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load the entire dataset with augmented data
data_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42)

# Calculate the number of validation samples
validation_samples = data_generator.samples - train_samples

# Split the data into training and validation sets
train_generator = tf.keras.preprocessing.image.DirectoryIterator(
    data_generator.directory,
    data_generator.image_data_generator,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset='training')

validation_generator = tf.keras.preprocessing.image.DirectoryIterator(
    data_generator.directory,
    data_generator.image_data_generator,
    target_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset='validation')

# Set the number of layers to fine-tune
fine_tune_layers = 10

# Unfreeze the last `fine_tune_layers` layers for training
for layer in model.layers[-fine_tune_layers:]:
    layer.trainable = True

# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model again after modifying the layers
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Define a custom learning rate
learning_rate = 0.0001

# Compile the model with the custom learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

# Save the trained model
model.save('plant_disease_model.h5')


Found 2552 images belonging to 2 classes.
Found 2552 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Epoch 1/10
 80/250 [========>.....................] - ETA: 53:06 - loss: 0.3590 - accuracy: 0.8973

250/250 [==============================] - 1504s 6s/step - loss: 0.3590 - accuracy: 0.8973


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model

# Load the pre-trained model
model = load_model('plant_disease_model.h5')

# Define the class labels
class_labels = ['disease', 'healthy']

def preprocess_image(image_path):
    # Load the image from the given path
    img = load_img(image_path, target_size=(224, 224))

    # Convert the image to a numpy array
    img_array = img_to_array(img)

    # Expand the dimensions to match the model input requirements
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the image (e.g., mean subtraction)
    preprocessed_img = preprocess_input(img_array)

    return preprocessed_img

def predict_disease(image_path):
    # Preprocess the image
    preprocessed_img = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(preprocessed_img)

    # Get the predicted class label
    predicted_class = np.argmax(predictions)
    predicted_label = class_labels[predicted_class]

    return predicted_label

# Example usage:
image_path = '/content/Tomato leaf late blight (1).jpg'  # Replace with the actual image path
predicted_label = predict_disease(image_path)
print(f"The predicted label for the input image is: {predicted_label}")

1/1 [==============================] - 1s 613ms/step
The predicted label for the input image is: healthy
